# CODING CHALLENGE – Marketing Affinitätsscores

## Installation und Import

In [ ]:
!pip install pandas numpy scikit-learn matplotlib tensorflow flask requests

In [2]:
import pandas as pd
import numpy as np
import os
import sklearn
from sklearn.model_selection import train_test_split
from sklearn import metrics  
from sklearn import tree
import sklearn.ensemble
import sklearn.svm
import matplotlib.pyplot as plt
import tensorflow as tf
from joblib import dump, load
import requests

## Aufgabenteil A: Data Science

### Laden des Datensatzes

In [3]:
dataset_path = 'HUK_MA_CC'

In [4]:
def _data_path(name):
    return os.path.join(dataset_path, f'{name}.csv')

# the original customer id is used as the dataframe id
interest_df = pd.read_csv(_data_path('interesse'), sep=',', index_col='id')
age_sex_df = pd.read_csv(_data_path('alter_geschlecht'), sep=',', index_col='id')
rest_df = pd.read_csv(_data_path('rest'), sep=';', index_col='id')

In [5]:
#rest_df

In [6]:
# join the dataframe into one single dataframe
merged_df = interest_df.join(age_sex_df).join(rest_df)

### Explorative Datenanalyse

In [7]:
merged_df

,Interesse,Geschlecht,Alter,Fahrerlaubnis,Regional_Code,Vorversicherung,Alter_Fzg,Vorschaden,Jahresbeitrag,Vertriebskanal,Kundentreue
id,,,,,,,,,,,
1,1.0,Male,44,1,28.0,0,> 2 Years,Yes,40454.0,26.0,217
2,0.0,Male,76,1,3.0,0,1-2 Year,No,33536.0,26.0,183
3,1.0,Male,47,1,28.0,0,> 2 Years,Yes,38294.0,26.0,27
4,0.0,Male,21,1,11.0,1,< 1 Year,No,28619.0,152.0,203
5,0.0,Female,29,1,41.0,1,< 1 Year,No,27496.0,152.0,39
...,...,...,...,...,...,...,...,...,...,...,...
381105,0.0,Male,74,1,26.0,1,1-2 Year,No,30170.0,26.0,88
381106,0.0,Male,30,1,37.0,1,< 1 Year,No,40016.0,152.0,131
381107,0.0,Male,21,1,30.0,1,< 1 Year,No,35118.0,160.0,161


#### Regional_Code

In [8]:
regional_code_list = merged_df.Regional_Code.unique().tolist()

In [9]:
print("Regional Code: - Prozent der Kunden mit Interesse - Anzahl der Kunden mit Interesse")
for regional_code in regional_code_list:
    filtered_df = merged_df.query(f'Regional_Code == {regional_code}')
    n_entries = filtered_df.shape[0]
    filtered_df = filtered_df.query(f'Interesse == 1')
    n_entries_interest = filtered_df.shape[0]
    print(regional_code, " - ", f'{n_entries_interest/n_entries:.2%}', " - ", f'{n_entries_interest}')

Regional Code: - Prozent der Kunden mit Interesse - Anzahl der Kunden mit Interesse
28.0  -  18.72%  -  19917
3.0  -  12.77%  -  1181
11.0  -  11.28%  -  1041
41.0  -  12.18%  -  2224
33.0  -  9.93%  -  760
6.0  -  6.99%  -  439
35.0  -  12.46%  -  865
50.0  -  6.27%  -  642
15.0  -  7.20%  -  958
45.0  -  10.97%  -  615
8.0  -  9.61%  -  3257
36.0  -  7.99%  -  703
30.0  -  7.38%  -  900
26.0  -  7.23%  -  187
16.0  -  6.68%  -  134
47.0  -  8.77%  -  652
48.0  -  10.19%  -  477
19.0  -  16.29%  -  250
39.0  -  12.36%  -  574
23.0  -  15.31%  -  300
37.0  -  7.93%  -  436
5.0  -  11.57%  -  148
17.0  -  7.03%  -  184
2.0  -  7.11%  -  287
7.0  -  13.17%  -  432
29.0  -  12.34%  -  1365
46.0  -  10.29%  -  2032
27.0  -  7.40%  -  209
25.0  -  4.27%  -  107
13.0  -  9.02%  -  364
18.0  -  13.16%  -  678
20.0  -  11.52%  -  223
49.0  -  7.48%  -  137
22.0  -  6.34%  -  83
44.0  -  4.08%  -  33
0.0  -  8.61%  -  174
9.0  -  8.16%  -  253
31.0  -  10.05%  -  197
12.0  -  9.76%  -  312
34.0

Hier handelt es sich um ein kategorisches Feature. Der Wohnort (z.B. Land/Stadt) kann relevant für das Interesse des Kunden sein (siehe oben), für ML-Modelle dürfte das Feature in dieser Form aber hinderlich sein da keine Vergleichbarkeit vorhanden ist. Die Umwandlung in ein kategorisches Feature findet im nächsten Schritt statt. Für die binären Features (Geschlecht, Vertriebskanal, Fahrzeugalter und Vorschaden) kann man stattdessen auch eine reine Datentypumwandlung nutzen.

#### Vertriebskanal

In [10]:
vertriebskanal_list = merged_df.Vertriebskanal.unique().tolist()

In [11]:
print("Vertriebskanal: - Prozent der Kunden mit Interesse - Anzahl der Kunden mit Interesse")
for vertriebskanal in vertriebskanal_list:
    filtered_df = merged_df.query(f'Vertriebskanal == {vertriebskanal}')
    n_entries = filtered_df.shape[0]
    filtered_df = filtered_df.query(f'Interesse == 1')
    n_entries_interest = filtered_df.shape[0]
    print(vertriebskanal, " - ", f'{n_entries_interest/n_entries:.2%}', " - ", f'{n_entries_interest}')

Vertriebskanal: - Prozent der Kunden mit Interesse - Anzahl der Kunden mit Interesse
26.0  -  19.94%  -  15891
152.0  -  2.86%  -  3858
160.0  -  2.18%  -  475
124.0  -  18.91%  -  13996
14.0  -  10.13%  -  63
13.0  -  14.75%  -  275
30.0  -  11.06%  -  156
156.0  -  21.55%  -  2297
163.0  -  30.42%  -  880
157.0  -  26.84%  -  1794
122.0  -  17.32%  -  1720
19.0  -  8.56%  -  19
22.0  -  5.42%  -  18
15.0  -  8.78%  -  78
154.0  -  24.60%  -  1474
16.0  -  8.60%  -  45
52.0  -  10.90%  -  115
155.0  -  32.01%  -  395
11.0  -  8.98%  -  108
151.0  -  3.14%  -  122
125.0  -  12.38%  -  127
25.0  -  19.97%  -  369
61.0  -  9.67%  -  56
1.0  -  3.26%  -  35
86.0  -  12.50%  -  6
31.0  -  25.36%  -  160
150.0  -  24.36%  -  76
23.0  -  13.74%  -  58
60.0  -  10.25%  -  53
21.0  -  8.11%  -  12
121.0  -  29.69%  -  19
3.0  -  30.40%  -  159
139.0  -  9.09%  -  13
12.0  -  16.86%  -  132
29.0  -  12.57%  -  106
55.0  -  14.95%  -  189
7.0  -  11.39%  -  182
47.0  -  12.70%  -  8
127.0  -  7.

#### Anzahl der Klassen

In [12]:
interest_n = merged_df['Interesse'].sum()

In [13]:
n = merged_df['Interesse'].shape[0]

In [14]:
interest_n/n

np.float64(0.12256336113815208)

Nur 12.26% der Samples sind interessiert. Dies kann zu Overfitting auf die stark vertretene Klasse (nicht interessiert) führen und ist später bei der Auswahl der Metriken zu beachten, da diese klassenweise messen sollten. Bei den hier vorhandenen binären Klassen bieten sich Precision und Recall als Metriken an. Die Accuracy ist wegen der Unausgewogenheit der Klassen weniger geeignet. Mit Klassengewichten kann zu starker Fokus auf die negative Klasse verhindert werden.

### Datenvorverarbeitung

#### Entfernen von Features

In [15]:
#drop_features = ['Vertriebskanal', 'Regional_Code']
drop_features = [] # hier werden keine Features entfernt
merged_df = merged_df.drop(drop_features, axis=1)

#### Umwandeln von kategorischen Features

In [16]:
categorical_features = ['Alter_Fzg', 'Vertriebskanal', 'Regional_Code']
categorical_features = [feature for feature in categorical_features if feature in list(merged_df.columns)]

merged_df = pd.get_dummies(merged_df, prefix=categorical_features, columns=categorical_features, dtype=np.float32)


binary_features = ['Vorschaden', 'Geschlecht']
binary_features = [feature for feature in binary_features if feature in list(merged_df.columns)]
for binary_feature in binary_features:
    values = merged_df[binary_feature].unique()
    mapping = {values[i]: i for i in range(len(values))}
    merged_df.replace({binary_feature: mapping}, inplace=True)

C:\Users\Kolja\AppData\Local\Temp\ipykernel_9520\1660940321.py:12: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  merged_df.replace({binary_feature: mapping}, inplace=True)


In [17]:
#list(merged_df.columns)

#### Normalisieren von Features

In [18]:
# normalizing data into 0...1 range, not necessary for decision trees, but would be for neural networks
max_normalize_features = ['Jahresbeitrag', 'Kundentreue', 'Alter']

In [19]:
for feature in max_normalize_features:
    merged_df[feature] = merged_df[feature]/merged_df[feature].max()
    #merged_df[feature] = np.minimum(merged_df[feature], 1.) #       not necessary, but would be needed for inference

In [20]:
merged_df

,Interesse,Geschlecht,Alter,Fahrerlaubnis,Vorversicherung,Vorschaden,Jahresbeitrag,Kundentreue,Alter_Fzg_1-2 Year,Alter_Fzg_< 1 Year,...,Regional_Code_43.0,Regional_Code_44.0,Regional_Code_45.0,Regional_Code_46.0,Regional_Code_47.0,Regional_Code_48.0,Regional_Code_49.0,Regional_Code_50.0,Regional_Code_51.0,Regional_Code_52.0
id,,,,,,,,,,,,,,,,,,,,,
1,1.0,0,0.517647,1,0,0,0.074892,0.725753,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0,0.894118,1,0,1,0.062085,0.612040,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,0,0.552941,1,0,0,0.070893,0.090301,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0,0.247059,1,1,1,0.052982,0.678930,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,1,0.341176,1,1,1,0.050903,0.130435,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
381105,0.0,0,0.870588,1,1,1,0.055853,0.294314,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
381106,0.0,0,0.352941,1,1,1,0.074081,0.438127,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
381107,0.0,0,0.247059,1,1,1,0.065013,0.538462,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [21]:
# Unterteilung in Trainings- und Testdaten

In [22]:
train_df, test_df = train_test_split(merged_df, test_size=0.2)

In [23]:
def get_feature_target_data(df, as_numpy=False):
    feature_df = df.drop(['Interesse'], axis=1)
    target_df = df[["Interesse"]]
    if as_numpy:
        # fit expects a 1-D vector as target -> [:,0]
        return feature_df.to_numpy(), target_df.to_numpy()[:,0]
    return feature_df, target_df

In [24]:
# separate data into a feature and a label dataframe
feature_train, target_train = get_feature_target_data(train_df, as_numpy=True)
feature_test, target_test = get_feature_target_data(test_df, as_numpy=True)
feature_test_df, _ = get_feature_target_data(test_df, as_numpy=False)
n_train_samples = None
if n_train_samples is not None:
    feature_train, target_train = feature_train[:n_train_samples], target_train[:n_train_samples]

### Modellvergleich

In [50]:
feature_train.shape

(304887, 218)

Die relevantesten Optionen sind Decision Trees sowie Random Forest, Bagging, Boosting, Neuronale Netze sowie Support Vector Machines. Aufgrund der Vielzahl an Features (218) sind einzelne Entscheidungsbäume eher nicht sinnvoll. Random Forest, Bagging, Boosting nutzen jeweils viele, kleine Entscheidungsbäume. Bei Random Forests wird nur eine Teilmenge an Features und Daten für jeden Baum genutzt, Bagging nutzt grundsätzlich alle Features, aber für jeden Baum nur eine Teilmenge der Daten. Boosting passt die Gewichtung der Trainingsdaten für das Training des nächsten Prädiktors jeweils an.

Alle drei Methoden sind denkbar, wir nutzen zunächst einen Random Forest, da es hier zwischen den einzelnen Bäumen größere Unterschiede begen sollte als bei Bagging und es schneller trainieren sollte als Boosting, auch weil Klassengewichte mitgegeben werden können, während Boosting die unbalancierten Klassen durch die sich verändernden Gewichte erst während des Trainings anpassen muss.


Zudem wird unten ein kleines neronales Netz entworfen.

### Training

In [25]:
def evaluate(clf, feature, target):
    # we use accuracy, recall and precision as metrics, while recall should be the most important one to avoid false negatives
    y_pred = classifier.predict(feature)
    print("Accuracy:", metrics.accuracy_score(target, y_pred))
    print("Recall:", metrics.recall_score(target, y_pred))
    print("Precision:", metrics.precision_score(target, y_pred))
    CM = metrics.confusion_matrix(target, y_pred)
    TN = CM[0][0]
    FN = CM[1][0]
    TP = CM[1][1]
    FP = CM[0][1]
    print("True Negatives:", TN, "False Negatives:", FN, "True Positives:", TP, "False Positives:", FP)

In [26]:
target_train.shape

(304887,)

#### Random Forest

In [70]:
classifier = sklearn.ensemble.RandomForestClassifier(n_estimators=100, max_depth=13, class_weight='balanced')

In [71]:
classifier.fit(feature_train, target_train)

RandomForestClassifier(class_weight='balanced', max_depth=13)

In [72]:
evaluate(classifier, feature_train, target_train)

Accuracy: 0.7088691876006521
Recall: 0.917182102980159
Precision: 0.28563328135553645
True Negatives: 181871 False Negatives: 3093 True Positives: 34254 False Positives: 85669


In [73]:
evaluate(classifier, feature_test, target_test)

Accuracy: 0.7057017658943612
Recall: 0.9093239346363345
Precision: 0.2828853374090441
True Negatives: 45276 False Negatives: 849 True Positives: 8514 False Positives: 21583


In [74]:
#fig, ax = plt.subplots(1, 1)
#fig.set_size_inches(18.5, 10.5)
#abc = tree.plot_tree(classifier, ax=ax, fontsize=8, feature_names = list(feature_train.columns))
#plt.savefig('test.png')

In [75]:
feat_imp = classifier.feature_importances_
feature_train_df = get_feature_target_data(train_df)[0]
feat_imp = {list(feature_train_df.columns)[i]: feat_imp[i] for i in range(feat_imp.shape[0])}

In [76]:
{k: v for k, v in sorted(feat_imp.items(), key=lambda item: item[1])}

{'Vertriebskanal_6.0': np.float64(0.0),
 'Vertriebskanal_41.0': np.float64(0.0),
 'Vertriebskanal_49.0': np.float64(0.0),
 'Vertriebskanal_50.0': np.float64(0.0),
 'Vertriebskanal_57.0': np.float64(0.0),
 'Vertriebskanal_75.0': np.float64(0.0),
 'Vertriebskanal_84.0': np.float64(0.0),
 'Vertriebskanal_115.0': np.float64(0.0),
 'Vertriebskanal_126.0': np.float64(0.0),
 'Vertriebskanal_144.0': np.float64(0.0),
 'Vertriebskanal_82.0': np.float64(1.2454310242770533e-10),
 'Vertriebskanal_92.0': np.float64(8.525824181832812e-10),
 'Vertriebskanal_83.0': np.float64(4.1260730591047656e-09),
 'Vertriebskanal_117.0': np.float64(1.3337387017455119e-08),
 'Vertriebskanal_143.0': np.float64(3.294912157066578e-08),
 'Vertriebskanal_112.0': np.float64(6.613330011619048e-08),
 'Vertriebskanal_87.0': np.float64(7.765511769892596e-08),
 'Vertriebskanal_67.0': np.float64(8.05605871687144e-08),
 'Vertriebskanal_62.0': np.float64(9.694335623344316e-08),
 'Vertriebskanal_27.0': np.float64(1.017937102388585

In [78]:
dump(classifier, 'random_forest_classifier.joblib') 

['random_forest_classifier.joblib']

#### Neural Network

In [79]:
model = tf.keras.Sequential(
    [
        tf.keras.layers.Dense(feature_train.shape[-1], activation="relu", name="layer1"),
        tf.keras.layers.Dense(128, activation="relu", name="layer2"), 
        tf.keras.layers.Dense(32, activation="relu", name="layer3"), 
        tf.keras.layers.Dense(16, activation="relu", name="layer4"), 
        tf.keras.layers.Dense(1, activation="sigmoid", name="layer5"), 
    ]
)

In [80]:
model(feature_train)
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ layer1 (Dense)                       │ (304887, 218)               │          47,742 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ layer2 (Dense)                       │ (304887, 128)               │          28,032 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ layer3 (Dense)                       │ (304887, 32)                │           4,128 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ layer4 (Dense)                       │ (304887, 16)                │             528 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ layer5 (Dense)                       │ (304887, 1)                 │              17 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 80,447 (314.25 KB)

 Trainable params: 80,447 (314.25 KB)

 Non-trainable params: 0 (0.00 B)

In [81]:
model.compile(optimizer=tf.keras.optimizers.Adam(), loss='binary_crossentropy', metrics=[tf.keras.metrics.Recall(), tf.keras.metrics.Precision(), tf.keras.metrics.BinaryAccuracy()])

In [82]:
positive_weight = 3.
negative_weight = 1.
class_weight = {0: negative_weight, 1: positive_weight}

In [83]:
model.fit(feature_train, target_train, validation_data=(feature_test, target_test), batch_size=100, epochs=10, shuffle=True, class_weight=class_weight)

Epoch 1/10
3049/3049 ━━━━━━━━━━━━━━━━━━━━ 5s 1ms/step - binary_accuracy: 0.7715 - loss: 0.5137 - precision: 0.3081 - recall: 0.6987 - val_binary_accuracy: 0.7823 - val_loss: 0.3293 - val_precision: 0.3256 - val_recall: 0.7211
Epoch 2/10
3049/3049 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - binary_accuracy: 0.7848 - loss: 0.4942 - precision: 0.3281 - recall: 0.7231 - val_binary_accuracy: 0.7744 - val_loss: 0.3292 - val_precision: 0.3221 - val_recall: 0.7570
Epoch 3/10
3049/3049 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - binary_accuracy: 0.7837 - loss: 0.4941 - precision: 0.3306 - recall: 0.7348 - val_binary_accuracy: 0.8130 - val_loss: 0.3114 - val_precision: 0.3465 - val_recall: 0.5897
Epoch 4/10
3049/3049 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - binary_accuracy: 0.7933 - loss: 0.4902 - precision: 0.3375 - recall: 0.7079 - val_binary_accuracy: 0.7664 - val_loss: 0.3376 - val_precision: 0.3178 - val_recall: 0.7866
Epoch 5/10
3049/3049 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - binary_accuracy: 0.7928 - loss: 0.4881 -

## Aufgabenteil B: ML Engineering

"python rest_api_app.py" starts the app using flask. The code below sends an example request.

In [122]:
# here, we start the api. alternatively, it can be started using the Dockerfile
if os.name == 'nt':
    # starts command in an external console for windows
    ! start python rest_api_app.py
else:
    # for linux
    ! python rest_api_app.py & 

In [84]:
url = 'http://127.0.0.1:5000/predict'

In [85]:
sample_idx = 8

data_dict = {}
for column in feature_test_df.columns:
    data_dict[column] = [float(feature_test_df[column].iloc[sample_idx])]

# here, we already post normalized and pre-processed data
# in a real, application these steps would be run on the server
response = requests.post(url, json=data_dict)

# Check the HTTP response status code
if response.status_code == 200:
    print(response.json())
else:
    # Handle the case where the API request failed
    print(f'Error {response.status_code}: {response.text}')


{'Prediction': [0.0]}


Die Dockerfile befindet sich ebenfalls im Ordner. Standardmäßig wird Port 5000 genutzt.